# ResourceBound Pattern - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu biblioteki zastosowaniu wzorca `ResourceBound`, który ułatwia obsługę różnych stanów (ładowanie, błąd, sukces) w przypadku obsługi danych z zewnętrznych serwisów.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExNXRjMjNlbjJmYTNiMW45OW1iZmh2dnpzN2l1MmxsejB0eWlqNDNnMCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/6lIYVQa6srpVlsTU7u/giphy.gif" width="200" />

Klasa `Resource` jest przydatna, gdy chcemy przekazywać różne stany operacji do interfejsu użytkownika. Na przykład, gdy pobieramy dane z sieci, możemy używać `Resource` do informowania interfejsu użytkownika o stanie ładowania, sukcesie lub błędzie, a także dostarczyć odpowiednie dane i komunikaty w zależności od wyniku operacji. Jest to często stosowane w architekturze **MVVM** (Model-View-ViewModel), aby lepiej zarządzać stanem interfejsu użytkownika i przekazywać wyniki operacji do widoku.

W tej aplikacji również wykorzystamy  [**JSONPlaceholder**](https://jsonplaceholder.typicode.com/), tym razem z endpointem `comments`

Nasza aplikacja wymaga dostępu do internetu, aby go uzyskać musimy dodać deklarację uprawnienia w pliku konfiguracyjnym aplikacji, który nazywa się `AndroidManifest.xml`. To uprawnienie informuje system operacyjny Android, że aplikacja potrzebuje dostępu do internetu.

In [ ]:
<uses-permission android:name="android.permission.INTERNET"/>

Rozpocznijmy od dodania zależności
```kotlin
    implementation (libs.retrofit2.retrofit)
    implementation (libs.gson)
    implementation (libs.converter.gson)

// ViewModel
    implementation (libs.lifecycle.viewmodel)
// LiveData
    implementation (libs.lifecycle.livedata)
```

```kotlin
[versions]
agp = "8.5.2"
converterGson = "2.10.1"
gson = "2.10.1"
junit = "4.13.2"
junitVersion = "1.2.1"
espressoCore = "3.6.1"
appcompat = "1.7.0"
lifecycleViewmodel = "2.8.4"
material = "1.12.0"
activity = "1.9.1"
constraintlayout = "2.1.4"
retrofitVersion = "2.9.0"

[libraries]
converter-gson = { module = "com.squareup.retrofit2:converter-gson", version.ref = "converterGson" }
gson = { module = "com.google.code.gson:gson", version.ref = "gson" }
junit = { group = "junit", name = "junit", version.ref = "junit" }
ext-junit = { group = "androidx.test.ext", name = "junit", version.ref = "junitVersion" }
espresso-core = { group = "androidx.test.espresso", name = "espresso-core", version.ref = "espressoCore" }
appcompat = { group = "androidx.appcompat", name = "appcompat", version.ref = "appcompat" }
lifecycle-livedata = { module = "androidx.lifecycle:lifecycle-livedata", version.ref = "lifecycleViewmodel" }
lifecycle-viewmodel = { module = "androidx.lifecycle:lifecycle-viewmodel", version.ref = "lifecycleViewmodel" }
material = { group = "com.google.android.material", name = "material", version.ref = "material" }
activity = { group = "androidx.activity", name = "activity", version.ref = "activity" }
constraintlayout = { group = "androidx.constraintlayout", name = "constraintlayout", version.ref = "constraintlayout" }
retrofit2-retrofit = { module = "com.squareup.retrofit2:retrofit", version.ref = "retrofitVersion" }

[plugins]
android-application = { id = "com.android.application", version.ref = "agp" }


```

Następnie dodajmy model danych

In [ ]:
public class CommentResponseItem {
    private String body;
    private String email;
    private int id;
    private String name;
    private int postId;

    public CommentResponseItem(String body, String email, int id, String name, int postId) {
        this.body = body;
        this.email = email;
        this.id = id;
        this.name = name;
        this.postId = postId;
    }

    public String getBody() {
        return body;
    }

    public void setBody(String body) {
        this.body = body;
    }

    public String getEmail() {
        return email;
    }

    public void setEmail(String email) {
        this.email = email;
    }

    public int getId() {
        return id;
    }

    public void setId(int id) {
        this.id = id;
    }

    public String getName() {
        return name;
    }

    public void setName(String name) {
        this.name = name;
    }

    public int getPostId() {
        return postId;
    }

    public void setPostId(int postId) {
        this.postId = postId;
    }

    @Override
    public String toString() {
        return "CommentResponseItem{" +
                "body='" + body + '\'' +
                ", email='" + email + '\'' +
                ", id=" + id +
                ", name='" + name + '\'' +
                ", postId=" + postId +
                '}';
    }

    @Override
    public boolean equals(Object o) {
        if (this == o) return true;
        if (o == null || getClass() != o.getClass()) return false;
        CommentResponseItem that = (CommentResponseItem) o;
        return id == that.id &&
                postId == that.postId &&
                Objects.equals(body, that.body) &&
                Objects.equals(email, that.email) &&
                Objects.equals(name, that.name);
    }

    @Override
    public int hashCode() {
        return Objects.hash(body, email, id, name, postId);
    }
}

Następnie dodajmy interfejs

In [ ]:
public interface PlaceholderApi {
    @GET("comments")
    Call<List<CommentResponseItem>> comments();
}

Dodajmy instancję `Retrofit` oraz repozytorium.

In [ ]:
public class RetrofitInstance {
    private static final String BASE_URL = "https://jsonplaceholder.typicode.com/";
    private static volatile PlaceholderApi apiInstance;

    public static PlaceholderApi getApi() {
        if (apiInstance == null) {
            synchronized (RetrofitInstance.class) {
                if (apiInstance == null) {
                    apiInstance = new Retrofit.Builder()
                            .baseUrl(BASE_URL)
                            .addConverterFactory(GsonConverterFactory.create())
                            .build()
                            .create(PlaceholderApi.class);
                }
            }
        }
        return apiInstance;
    }
}

In [ ]:
public class CommentRepository {
    private final PlaceholderApi api = RetrofitInstance.getApi();

    public Call<List<CommentResponseItem>> getComments() {
        return api.comments();
    }
}

Zanim przejdziemy do viewmodelu, dodajmy klasę `Resource`, która pozwoli nam reprezentować różne stany operacji. Stan będziemy reprezentować za pomocą trybu wyliczeniowego.

In [ ]:
public enum Status {
    SUCCESS,
    ERROR,
    LOADING
}

In [ ]:
public class Resource<T> {
    @NonNull
    public final Status status;
    @Nullable
    public final T data;
    @Nullable public final String message;

    private Resource(@NonNull Status status, @Nullable T data, @Nullable String message) {
        this.status = status;
        this.data = data;
        this.message = message;
    }

    public static <T> Resource<T> success(@NonNull T data) {
        return new Resource<>(SUCCESS, data, null);
    }

    public static <T> Resource<T> error(String msg, @Nullable T data) {
        return new Resource<>(ERROR, data, msg);
    }

    public static <T> Resource<T> loading(@Nullable T data) {
        return new Resource<>(LOADING, data, null);
    }
}

- `public class Resource<T>`: Jest to deklaracja klasy `Resource`, która jest opakowaniem wyniku operacji. Parametr generyczny `T` oznacza typ danych, które będą przechowywane w obiekcie `Resource`.
- `@Nullable public final T data`: To jest pole `data`, które przechowuje wynik operacji. Może to być obiekt zawierający `dane` lub `null`, jeśli operacja nie zwróciła wyniku.
- `@Nullable public final String message`: To jest pole `message`, które może zawierać wiadomość lub komunikat związany z wynikiem operacji. Jest to przydatne do przechowywania informacji zwrotnych lub błędów.
- `public static <T> Resource<T> success(@NonNull T data)`: Jest to funkcja `Resource`, która reprezentuje sukces operacji. Przyjmuje dane (`data`) jako argument.
- `public static <T> Resource<T> error(String msg, @Nullable T data)`: Jest to funkcja `Resource`, która reprezentuje błąd operacji. Przyjmuje wiadomość błędu (`message`) i opcjonalnie dane związane z błędem (`data`) jako argumenty.
- `public static <T> Resource<T> loading(@Nullable T data)`: Jest to funkcja `Resource`, która reprezentuje stan ładowania. Nie przyjmuje żadnych danych. Jest używana do informowania interfejsu użytkownika, że operacja jest w trakcie wykonywania.

Następnie przejdźmy do implementacji viewmodel.

In [ ]:
public class CommentsViewModel extends ViewModel {

    private final CommentRepository repository = new CommentRepository();
    private final MutableLiveData<Resource<List<CommentResponseItem>>> _comments = new MutableLiveData<>();
    public LiveData<Resource<List<CommentResponseItem>>> comments = _comments;

    public CommentsViewModel() {
        try {
            getPosts();
        } catch (IOException e) {
            throw new RuntimeException(e);
        }
    }

    private void getCommentsList() throws IOException {
        Call<List<CommentResponseItem>> response = repository.getComments();

        response.enqueue(new Callback<List<CommentResponseItem>>() {
            @Override
            public void onResponse(@NonNull Call<List<CommentResponseItem>> call, @NonNull Response<List<CommentResponseItem>> response) {
                if (response.isSuccessful()){
                    _comments.postValue(Resource.loading(null));
                    List<CommentResponseItem> commentsResponse = response.body();
                    if (postsResponse != null)
                        _comments.postValue(Resource.success(commentsResponse));
                }
            }

            @Override
            public void onFailure(@NonNull Call<List<CommentResponseItem>> call, @NonNull Throwable t) {
                _comments.postValue(Resource.error(t.getLocalizedMessage(), null));
            }
        });
    }
}

Funkcja `getCommentsList()` jest odpowiedzialna za inicjowanie operacji pobierania listy komentarzy, ustawianie odpowiedniego stanu `_comments` (`Loading`), wykonanie żądania HTTP, przetworzenie odpowiedzi i zaktualizowanie stanu `_comments` w zależności od wyniku operacji.

- `_comments.postValue(Resource.loading(null));`: Na początku funkcja ustawia stan `_comments` na `Resource.loading()`, co oznacza, że operacja pobierania jest w trakcie wykonywania. Jest to sygnał dla interfejsu użytkownika, że aplikacja jest w trakcie ładowania danych.
- `_comments.postValue(Resource.success(postsResponse));`: Jeśli odpowiedź jest udana, to wykorzystywany jest warunek `if (postsResponse != null)`, który sprawdza, czy ciało odpowiedzi (`response.body()`) nie jest `null`, a następnie tworzy obiekt za pomocą funcji `Resource.success` z danymi komentarzy i go zwraca. Ten obiekt oznacza sukces operacji i zawiera dane komentarzy.
- `_comments.postValue(Resource.error(t.getLocalizedMessage(), null));`: Jeśli odpowiedź nie jest udana (kod stanu HTTP jest różny od 2xx), to jest tworzony obiekt przez funkcję `Resource.error` z komunikatem błędu uzyskanym z odpowiedzi (`t.getLocalizedMessage()`). Ten obiekt oznacza błąd operacji.

Przejdźmy do fragmentu oraz elementów `RecyclerView`.

In [ ]:
public class CommentViewHolder extends RecyclerView.ViewHolder {
    private final RvItemBinding binding;

    public CommentViewHolder(RvItemBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(CommentResponseItem item) {
        binding.title.setText(item.getName());
        binding.body.setText(item.getBody());
        binding.commentId.setText(String.valueOf(item.getPostId()));
    }
}

In [ ]:
public class CommentComparator extends DiffUtil.ItemCallback<CommentResponseItem> {
    @Override
    public boolean areItemsTheSame(@NonNull CommentResponseItem oldItem, @NonNull CommentResponseItem newItem) {
        return oldItem == newItem;
    }

    @Override
    public boolean areContentsTheSame(CommentResponseItem oldItem, @NonNull CommentResponseItem newItem) {
        return oldItem.equals(newItem);
    }
}

In [ ]:
public class CommentAdapter extends ListAdapter<CommentResponseItem, CommentViewHolder> {
    public CommentAdapter(CommentComparator userComparator) {
        super(userComparator);
    }

    @NonNull
    @Override
    public CommentViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        RvItemBinding binding = RvItemBinding.inflate(LayoutInflater.from(parent.getContext()), parent, false);
        return new CommentViewHolder(binding);
    }

    @Override
    public void onBindViewHolder(CommentViewHolder holder, int position) {
        CommentResponseItem item = getItem(position);
        holder.bind(item);
    }
}

In [ ]:
public class CommentsFragment extends Fragment {

    private FragmentCommentsBinding binding;
    private CommentsViewModel viewModel;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container, Bundle savedInstanceState) {
        binding = FragmentCommentsBinding.inflate(inflater, container, false);

        CommentAdapter commentAdapter = new CommentAdapter(new CommentComparator());
        viewModel = new ViewModelProvider(this).get(CommentsViewModel.class);

        viewModel.comments.observe(getViewLifecycleOwner(), response -> {
            if (response.status == Status.SUCCESS) {
                    List<CommentResponseItem> data = response.data;
                    if (data != null) {
                        commentAdapter.submitList(data);
                    }
                } else if (response.status == Status.ERROR) {
                    // Obsługa błędu
                } else if (response.status == Status.LOADING) {
                    // Obsługa stanu ładowania
                }
        });

        binding.recycler.setLayoutManager(new LinearLayoutManager(requireContext()));
        binding.recycler.setAdapter(commentAdapter);

        return binding.getRoot();
    }
}

- `if (response.status == Status.SUCCESS) { ... }`: Jeśli stan `response` to `Resource.success`, to dostarczana jest lista komentarzy dostępną w `response.data`.
- `else if (response.status == Status.ERROR)`: Jeśli stan `response` to `Resource.error`, to nie są podejmowane żadne działania. Tutaj można dodać obsługę błędów, np. wyświetlanie komunikatu o błędzie.
- `else if (response.status == Status.LOADING)`: Jeśli stan `response` to `Resource.loading`, to nie są podejmowane żadne działania. To może być miejsce na wyświetlenie wskaźnika ładowania lub innej informacji o trwającym procesie ładowania.

Możemy przetestować aplikację.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExNXRjMjNlbjJmYTNiMW45OW1iZmh2dnpzN2l1MmxsejB0eWlqNDNnMCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/6lIYVQa6srpVlsTU7u/giphy.gif" width="200" />